
<a id='chap-tpintroductionapprentissagesupervise'></a>

# Travaux pratiques - Introduction à l’apprentissage supervisé

**L’objectif** de cette première séance de travaux pratiques est de vous faire découvrir les problèmes de classification et les problèmes de régression à l'aide de modèles simples : régressions linéaires et polynomiales et arbres de décision.

Références externes utiles :
- [Documentation NumPy](https://docs.scipy.org/doc/numpy/user/index.html)  
- [Documentation SciPy](https://docs.scipy.org/doc/scipy/reference/)  
- [Documentation MatPlotLib](http://matplotlib.org/)  
- [Documentation de scikit-learn](http://scikit-learn.org/stable/index.html)  

In [ ]:
# Import des bibliothèques utiles
import matplotlib.pyplot as plt
import numpy as np
import sklearn

## Régression avec modèles linéaires et polynomiaux

Pour commencer ce premier TP, nous examinerons un problème simple de
régression avec une seule variable prédictive, similaire à celui
illustré dans le support de cours. À partir d’un ensembles
d’observations $ \{(x_1, y_1), (x_2, y_2), \dots, (x_n, y_n)\} $,
l’objectif est de trouver la fonction $ \hat{f} $ telle que
$ \hat{f}(x_i) = y_i $.

Pour cet exemple, commençons par générer des données synthétiques. En
pratique, nous n’aurions bien entendu pas à réaliser ce travail : le jeu
de données est constitué des observations réelles. Notre jeu de données
d’exemple consiste en une portion de sinusoïde sur laquelle un bruit
gaussien a été appliqué.

In [ ]:
# Fixe l'aléas pour pouvoir reproduire les résultats
np.random.seed(0)
# On tire au hasard 100 points dans l'intervalle [0,2]
X = 2 * np.random.rand(100, 1)
# On calcule la valeur de sin(x) pour chaque point, plus un bruit gaussien aléatoire
y = np.sin(X[:,0]) + 0.15 * np.random.randn(100)

Nous pouvons visualisation toutes les données générées, qui forment
notre matrice d’observation pour ce problème de régression :

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X, y, s=50)
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.show()

Nous générons maintenant un premier découpage entre données d’apprentissage et données de test. Ce découpage est aléatoire et
s’effectue à l’aide de la fonction [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
de scikit-learn. Le paramètre `test_size` permet de spécifier le pourcentage du jeu de données qui sera contenu dans le jeu de test.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
plt.figure(figsize=(8, 6))
plt.scatter(X_train, y_train, s=50, edgecolors='none', label="Exemples d'entraînement")
plt.scatter(X_test, y_test, c='none', s=50, edgecolors='blue', label="Exemples d'évaluation")
plt.legend()
plt.show()

Nous cherchons d’abord un modèle linéaire pour ce problème de régression. Regardez la classe
[LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)
de scikit-learn (ainsi que [ces explications](http://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares)).
Dans le cas présent, scikit-learn va chercher la fonction affine $ f: x \rightarrow y = ax + b $ en déterminant les paramètres
$ a $ et $ b $ à l’aide de la méthode des moindres carrés.

Les résultats sont évalués ici à travers le **coefficient de détermination**, qui est le rapport entre la variance expliquée par le
modèle et la variance totale (de la variable expliquée), voir [les explications](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html).

Commençons par instancier un modèle de régression linéaire.

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

Tous les modèles implémentés dans scikit-learn suivent la même interface et présentent au moins les trois méthodes suivantes :

- `.fit()` permet d’apprendre le modèle en estimant ses paramètres à
  partir d’un jeu de données,  
- `.predict()` ou `.transform()` permet d’appliquer le modèle à de
  nouvelles données,  
- `.score()` permet d’évaluer le modèle selon un critère prédéfini
  (taux de bonne classification, coefficient de détermination, etc.)
  sur un jeu de test.  


Par exemple, dans notre cas, la méthode `.fit(X,y)` permet de déterminer les paramètres de la régression linéaire à partir de nos
observations `X` et de notre vérité terrain `y`.

In [ ]:
# optimisation du modèle: détermination des paramètres de la régression linéaire par la méthode des moindres carrés
reg.fit(X_train, y_train)

Nous pouvons évaluer à quel point le modèle « colle » aux données à l’aide du coefficient de détermination calculé sur les exemples du jeu
d’apprentissage :

In [ ]:
# attention, score() ici ne renvoie pas l'erreur mais la valeur du coefficient de détermination R² !
coeff_train = reg.score(X_train, y_train)
print(f"Coefficient de détermination R² en train : {coeff_train:.2f}")

## Question:

Calculer le coefficient de détermination R² sur le jeu de test. Que constatez-vous ?

Nous pouvons ensuite, à partir des coefficients qui ont été estimés,
tracer notre modèle de régression :

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X_train,y_train, s=50, edgecolors='none', label="Exemples d'apprentissage")
plt.scatter(X_test,y_test, c='none', s=50, edgecolors='blue', label="Exemples d'évaluation")
plt.xlabel("$x$")
plt.ylabel("$y$")

x_min, x_max = plt.xlim()
nx = 100
xx = np.linspace(x_min, x_max, nx).reshape(-1,1)
plt.plot(xx,reg.predict(xx), color='k', label="Régression linéaire")
plt.title("Régression linéaire unidimensionnelle")
plt.legend()
plt.show()

## Question

Calculez l’erreur quadratique moyenne du modèle sur les données d’apprentissage et ensuite sur les données de test. Vous pouvez
l’implémenter vous-même à l’aide de NumPy ou bien consulter la [documentation du module
sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics).

### Régression polynomiale

La régression linéaire est un outil pratique mais quelque peu limité : les relations entre les variables explicatives et la variable à prédire
sont rarement linéaires en pratique ! Nous pourrions chercher dans une famille paramétrique plus grande, par exemple les polynômes à une seule
variable de degré $ d $ : $ P : x \rightarrow a_0 + a_1 x + a_2 x^2 + \dots + a_d x^d $.

La visualisation interactive ci-dessous permet de visualiser les résultats d’une régression polynomiale appliquée sur notre jeu de
données, de tracer la régression et d’afficher les coefficients de détermination R² en apprentissage et en test.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.pipeline import Pipeline

from ipywidgets import interact, fixed

@interact(degree=(0,10,1), show=fixed(True))
def polynomial_fit(degree, show=False):
    pipe = Pipeline([('poly', PolynomialFeatures(degree=degree)),
                    ('reg', linear_model.LinearRegression())])

    pipe.fit(X_train, y_train)
    train_score = pipe.score(X_train, y_train)
    test_score = pipe.score(X_test, y_test)

    if show:
        plt.figure(figsize=(8, 6))
        plt.scatter(X_train, y_train, s=50, edgecolors='none', label="Exemples d'apprentissage")
        plt.scatter(X_test, y_test, c='none', s=50, edgecolors='blue', label="Exemples de test")
        x_min, x_max = plt.xlim()
        xx = np.linspace(x_min, x_max, nx).reshape(-1,1)
        plt.plot(xx,pipe.predict(xx),color='black', label="Régression polynomiale")
        plt.legend()
        plt.show()

        print(f"Coefficient R² (train): {train_score:.3f}")
        print(f"Coefficient R² (test): {test_score:.3f}")
    return train_score, test_score

### Question

Que se passe-t-il lorsque l’on modifie le degré du polynôme ? Quelles observations pouvez-vous faire concernant le sous-apprentissage ? Le sur-apprentissage ?

### Question

*(pour aller plus loin)* Tracer les courbes du coefficient R² sur le jeu d’apprentissage et sur le jeu de test en fonction du degré du polynôme (entre 1 et 10). Vous
pouvez utiliser la fonction `polynomial_fit` pour ce faire qui renvoie un tuple contenant les deux scores. Que constatez-vous ?

## Classification avec modèles linéaires et non-linéaires

Voyons maintenant comment les notions de sur-apprentissage et sous-apprentissage peuvent se manifester dans le cas d’un problème de
classement (ou classification). Nous examinerons un problème simple de discrimination entre deux classes, similaire à celui illustré dans le
support de cours.

Nous générons des données à partir de lois normales bidimensionnelles. Pour la première classe nous employons une seule loi avec des variances
différentes et une rotation qui introduit des covariances. La seconde classe est un mélange de 4 lois normales avec des centres différents.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# fixer la graine aléatoire de numpy permet d'obtenir systématiquement les mêmes résultats
np.random.seed(150)

from sklearn import datasets

dataset = "gaussian"

if dataset == "gaussian":
    # définir matrices de rotation et de dilatation
    rot = np.array([[0.94, -0.34], [0.34, 0.94]])
    sca = np.array([[3.4, 0], [0, 2]])
    # générer données classe 1
    c1d = (np.random.randn(200,2)).dot(sca).dot(rot)

    # générer données classe 2
    c2d1 = np.random.randn(25,2)+[-10, 2]
    c2d2 = np.random.randn(25,2)+[-7, -2]
    c2d3 = np.random.randn(25,2)+[-2, -6]
    c2d4 = np.random.randn(25,2)+[5, -7]

    data = np.concatenate((c1d, c2d1, c2d2, c2d3, c2d4))

    # générer étiquettes de classe
    l1c = np.ones(200, dtype=int)
    l2c = np.zeros(100, dtype=int)
    labels = np.concatenate((l1c, l2c))
elif dataset == "iris":
    iris_X, iris_y = datasets.load_iris(return_X_y=True)
    data = iris_X[:, :2]
    labels = iris_y
elif dataset == "wine":
    wine_X, wine_y = datasets.load_wine(return_X_y=True)
    data = wine_X[:, (0,10)]
    labels = wine_y

Visualisation de toutes les données :

In [ ]:
# les échantillons du premier groupe sont en rouge 'r', ceux du deuxième groupe en vert ("green") 'g'
cmp = np.array(['r','g', 'b'])
plt.figure(figsize=(8, 6))
plt.scatter(data[:,0],data[:,1], c=cmp[labels], s=50, edgecolors='none')
plt.show()

Nous générons maintenant un premier découpage entre données d’apprentissage et données de test. Les données de test sont affichées
avec cercles vides (`c='none'`), les données d’apprentissage avec des cercles remplis.

In [ ]:
from sklearn.model_selection import train_test_split

# découpage des données en train et test
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=0)

plt.figure(figsize=(8, 6))
plt.scatter(X_train[:,0],X_train[:,1],c=cmp[y_train], s=50, edgecolors='none', label="Exemples d'apprentissage")
plt.scatter(X_test[:,0], X_test[:,1], c='none' ,s=50, edgecolors=cmp[y_test], label="Exemples de test")
plt.legend()
plt.show()

## Question

Combien d’échantillons le jeu de données d’apprentissage contient-il ? La [documentation de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
peut vous aider.

### Modèle linéaire : SVM

Nous cherchons d’abord un modèle linéaire pour ce problème de discrimination entre deux classes et utilisons pour cela une machine à vecteur de support (SVM). En résumé, cette méthode cherche la frontière de décision linéaire qui sépare au mieux les données.

In [ ]:
from sklearn.svm import LinearSVC
svm = LinearSVC()

# évaluation et affichage sur split1
svm.fit(X_train, y_train)
print("Le score sur le jeu d'apprentissage est de : {:.3f}".format(svm.score(X_train, y_train)))

print("Le score sur le jeu de test est de : {:.3f}".format(svm.score(X_test, y_test)))

### Question

À l’aide de la documentation de scikit-learn, déterminez à quoi correspond la valeur renvoyée par la méthode .score().

Nous pouvons examiner visuellement le modèle trouvé (la frontière de
discrimination linéaire, c’est-à-dire ici une droite dans le plan) :

In [ ]:
# on créé une nouvelle figure sur laquelle on affiche les points
plt.figure(figsize=(8, 6))
plt.scatter(X_train[:,0], X_train[:,1], c=cmp[y_train], s=50, edgecolors='none', label="Train")
plt.scatter(X_test[:,0],  X_test[:,1], c='none', s=50, edgecolors=cmp[y_test], label="Test")

# on calcule pour chaque point du plan sa probabilité d'appartenir à chaque classe
nx, ny = 400, 400
x_min, x_max = plt.xlim()
y_min, y_max = plt.ylim()
# meshgrid permet d'échantillonner tous les points du plan (entre x_min et x_max)
xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),np.linspace(y_min, y_max, ny))
# .decision_function permet d'obtenir le "score" assigné par la SVM à chaque observation
Z = svm.decision_function(np.c_[xx.ravel(), yy.ravel()])
#for cls_idx in range(max(labels)):
zz = Z.reshape(xx.shape)
# on dessine la frontière correspond à un score de 0,5
# les scores < 0,5 correspondent à la classe 0
# les scores > 0,5 correspondent à la classe 1
plt.contour(xx, yy, zz, [0.5])
plt.legend()
plt.show()

## Question

À l'aide de la [documentation de scikit-learn](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics), calculez la matrice de confusion du modèle, ainsi que son exactitude (*accuracy*).

Au-delà de quelle performance peut-on considérer que le modèle a appris quelque chose ?

Comme le jeu de de données est déséquilibré (il y a plus de données de la classe 2 que de la classe 1), la baseline à battre est un modèle qui prédirait systématiquement la classe majoritaire, ici la classe 2 : ce modèle obtiendrait ~66% d'*accuracy*.

### Modèle non-linéaire: arbre de classification

Comme vous avez pu le constater, la frontière de séparation idéale entre les deux groupes n’est pas linéaire. Il nous faut donc changer de
famille paramétrique pour en choisir une de plus grande capacité. Nous allons donc utiliser un arbre de décision pour réaliser la classification binaire. Nous utilisons
pour cela la classe [MLPClassifier](https://scikit-learn.org/stable/modules/tree.html)
de scikit-learn. Il est très
instructif d’examiner [les valeurs par défaut des paramètres](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier).

Il n’est pas indispensable de connaître le fonctionnement précis des arbres de décision pour ce TP, toutefois si le sujet vous intéresse vous pouvez consulter [ce cours](https://cedric.cnam.fr/vertigo/Cours/ml2/coursArbresDecision.html).

Le principal paramètre que nous allons manipuler est la profondeur maximale de l'arbre. Par défaut, scikit-learn ne limite pas la profondeur et l'arbre peut être aussi profond que nécessaire pour que toutes les feuilles ne contiennent que des observations de la même classe.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=None)

# évaluation et affichage sur split1
clf.fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
print(f"Le score en train est {train_score:.3f}")

test_score = clf.score(X_test, y_test)
print(f"Le score en test est {test_score:.3f}")

In [ ]:
# créer une nouvelle figure
plt.figure()
# afficher les nuages de points apprentissage (remplis) et de test (vides)
plt.scatter(X_train[:,0],X_train[:,1],c=cmp[y_train],s=50, edgecolors='none', label="Train")
plt.scatter(X_test[:,0], X_test[:,1], c='none', s=50, edgecolors=cmp[y_test], label="Test")

# calculer la probabilité de sortie du perceptron pour tous les points du plan
nx, ny = 200, 200
x_min, x_max = plt.xlim()
y_min, y_max = plt.ylim()
xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),np.linspace(y_min, y_max, ny))
Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])
for cls_idx in range(max(labels)):
    zz = Z[:, cls_idx].reshape(xx.shape)
    # on dessine la frontière correspond à un score de 0,5
    # les scores < 0,5 correspondent à la classe 0
    # les scores > 0,5 correspondent à la classe 1
    plt.contour(xx, yy, zz, [0.5])
plt.legend()
plt.show()

### Question

Refaites l’expérience avec `max_depth = 2`. Dans quel cas la régularisation est plus forte ? Quelle est la conséquence sur les résultats ?

In [ ]:
import ipywidgets as widgets

@interact(max_depth=widgets.IntSlider(min=1, max=10, step=1))
def tree_fit(max_depth):
    clf = DecisionTreeClassifier(max_depth=max_depth)

    # évaluation et affichage sur split1
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    print(f"Le score en train est {train_score:.3f}")

    test_score = clf.score(X_test, y_test)
    print(f"Le score en test est {test_score:.3f}")

    # on créé une nouvelle figure sur laquelle on affiche les points
    plt.figure(figsize=(8, 6))
    plt.scatter(X_train[:,0], X_train[:,1], c=cmp[y_train], s=50, edgecolors='none', label="Train")
    plt.scatter(X_test[:,0],  X_test[:,1], c='none', s=50, edgecolors=cmp[y_test], label="Test")

    # on calcule pour chaque point du plan sa probabilité d'appartenir à chaque classe
    nx, ny = 400, 400
    x_min, x_max = plt.xlim()
    y_min, y_max = plt.ylim()
    # meshgrid permet d'échantillonner tous les points du plan (entre x_min et x_max)
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, nx),np.linspace(y_min, y_max, ny))
    # .predict_proba permet de prédire le score de chaque classe pour un ensemble d'observations
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    for cls_idx in range(max(labels)):
        zz = Z[:, cls_idx].reshape(xx.shape)
        # on dessine la frontière correspond à un score de 0,5
        # les scores < 0,5 correspondent à la classe 0
        # les scores > 0,5 correspondent à la classe 1
        plt.contour(xx, yy, zz, [0.5])
    plt.legend()
    plt.show()

### Question

Expérimentez avec la visualisation ci-dessus. Que se passe-t-il lorsque la profondeur augmente ? Lorsqu'elle diminue ?
Comment pouvez-vous interpréter ces résultats du point de vue de la capacité du modèle, du sur-apprentissage et du sous-apprentissage ?

## Question

À l'aide de la fonction [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report) de scikit-learn, affichez les performances de classification du modèle obtenu. Quelles sont les métriques utilisées par scikit-learn ? À quoi correspondent-elles ?